# Forensics - A. Get Started 1 - Provision DSVM with Azure Forensics Tools

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0,  azure-cli-profile 2.1.4<br>
__OS Supported:__<br>
    -  Linux
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The notebook provides sample code to use create a DSVM, install Mono and pythonnet, then use Azure custom script extension to deploy our Azure Forensics libraries on onto the newly created DSVM.

<font color=red>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. If you don't have a DSVM yet, you may use the following instructions to create one 
2. Install Mono, Pythonnet and dependency to enable Azure Forensics Functionalities
3. Deploy Azure Forensics libraries through VM Extension

## 1. If you don't have a DSVM yet, you may use the following instructions to create one

1. Authentication

In [ ]:
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant ''

In [ ]:
# User Input for creating a new DSVM
vm_size = 'Standard_DS3_v2'

# replace [[your_subcription_id]] with 'real subscription id'
!az account set --subscription [[your_subcription_id]]

In [ ]:
# replace all [[your_stuff]] with 'real values'
!az group deployment create \
    --resource-group [[resource_group]] \
    --template-uri https://raw.githubusercontent.com/Azure/DataScienceVM/master/Scripts/CreateDSVM/Ubuntu/azuredeploy.json \
    --parameters \
    '{ \
      "$schema": "https://schema.management.azure.com/schemas/2015-01-01/deploymentParameters.json#",\
      "contentVersion": "1.0.0.0",\
      "parameters": {\
         "adminUsername": { "value" : "[[admin_user_id]]"},\
         "adminPassword": { "value" : "[[admin_user_password]]"},\
         "vmName": { "value" : "[[vm_name]]"},\
         "vmSize": { "value" : "Standard_DS3_v2"}\
      }\
    }'

## *** Please go to the project page to select the VM that you just created as your new computing platform (Run on ...)  to continue ...

## 2. Install Mono, Pythonnet and dependencies to enable Azure Forensics Functionalities

In [ ]:
# Uncomment the code, run it only once for the lifetime of the DSVM
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv-keys 3FA7E0328081BFF6A14DA29AA6A19B38D3D831EF
!sudo apt install -y apt-transport-https ca-certificates
!echo "deb https://download.mono-project.com/repo/ubuntu stable-xenial main" | sudo tee /etc/apt/sources.list.d/mono-official-stable.list
!sudo apt update
!sudo apt install -y mono-devel

In [ ]:
# Uncomment the code, run it only once for the lifetime of the DSVM
!sudo apt-get install -y clang
!sudo apt-get install -y libglib2.0-dev
!sudo apt-get install python-dev

In [ ]:
# Uncomment the code, run it only once for the lifetime of the DSVM
!git clone https://github.com/pythonnet/pythonnet
!python pythonnet/setup.py install

## 3. Deploy Azure Forensics libraries through VM Extension

In [ ]:
# For newly created DSVM please install required packages once
!pip install azure

In [ ]:
# For newly created DSVM please install required packages once
!pip install azure-cli-profile

In [ ]:
# For newly created DSVM please install required packages once
!pip install --upgrade Azure-Sentinel-Utilities

In [ ]:
# Initialization
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.storage import StorageManagementClient
from azure.common.client_factory import get_client_from_cli_profile
import SentinelAzure
from SentinelWidgets import *
import os

In [ ]:
# User Input and Save to Environmental store
# Set to True if you want to reset the environmental values
reset_environment_valiables = False
env_dir = %env
env_list = ['tenant_domain', 'subscription_id', 'resource_group']
envs =  {env_list[0]:'', env_list[1]:'', env_list[2]:''}
envs = WidgetViewHelper.set_env(reset_environment_valiables, env_dir, envs)
tenant_domain = envs[env_list[0]]
subscription_id = envs[env_list[1]]
resource_group = envs[env_list[2]]

vm_admin_user_id = input('vm admin user id:')

In [ ]:
# Authenticate to Azure Resource Management on the new DSVM
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant $tenant_domain

compute_client = get_client_from_cli_profile(ComputeManagementClient, subscription_id = subscription_id)
storage_client = get_client_from_cli_profile(StorageManagementClient, subscription_id = subscription_id)
compute = SentinelAzure.azure_compute_helper.ComputeHelper(compute_client, resource_group)
storage = SentinelAzure.azure_storage_helper.StorageHelper(storage_client)

In [ ]:
# preparing vm_extension_properties
command_to_execute, file_list = WidgetViewHelper.get_vm_extension_properties('DSVM', None, vm_admin_user_id)
vm_extension_properties = SentinelAzure.LinuxVMExtensionProperties(command_to_execute, file_list)

In [ ]:
# Select the newly created DSVM
selected_vm = WidgetViewHelper.select_vm(compute)
display(selected_vm)

In [ ]:
# Input for VM and new VM extension
vm_extension_name = 'azureforensics'
vm_name = selected_vm.value
vm_location = compute.get_vm(vm_name).location

In [ ]:
# Create VM Extension
vm_extension = compute.initialize_vm_extension(vm_extension_properties, vm_location)
create_vm_async = compute.create_vm_extension_async(vm_name, vm_extension_name, vm_extension)
print(create_vm_async.provisioning_state)